## IMPORTAR LAS LIBRERIAS

Actualizar a las que se usen finalmente en tu proyecto.

In [1]:
import numpy as np
import pandas as pd
import pickle

#Automcompletar rápido
%config IPCompleter.greedy=True

from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import Binarizer
from sklearn.preprocessing import MinMaxScaler

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import HistGradientBoostingRegressor

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline

## CARGAR LOS DATOS

### Ruta del proyecto

In [2]:
ruta_proyecto = 'C:/Users/isaac/Google Drive/DS4B/CursoMachineLearningPython/03_MACHINE_LEARNING/08_CASOS/03_RIESGOS'

### Nombre del fichero de datos

In [3]:
nombre_fichero_datos = 'prestamos.csv'

### Cargar los datos

In [4]:
ruta_completa = ruta_proyecto + '/02_Datos/01_Originales/' + nombre_fichero_datos

df = pd.read_csv(ruta_completa,index_col=0)

### Seleccionar solo las variables finales

#### Apuntar (manualmente) la lista de variables finales sin extensiones

In [5]:
variables_finales = ['ingresos_verificados',
                     'vivienda',
                     'finalidad',
                     'num_cuotas',
                     'antigüedad_empleo',
                     'rating',
                     'ingresos',
                     'dti',
                     'num_lineas_credito',
                     'porc_uso_revolving',
                     'principal',
                     'tipo_interes',
                     'imp_cuota',
                     'num_derogatorios',
                     'estado',
                     'imp_amortizado',
                     'imp_recuperado'
                  ]

#### Crear la matriz de variables procesos (excel)

Ir a la plantilla de Excel "Fase Producción Plantilla Procesos" y crear la matriz de variables por procesos.

#### Actualizar las importaciones

Ir arriba a la celda de importacion de paquetes y actualizarlos con los que finalmente vamos a usar.

## ESTRUCTURA DE LOS DATASETS

### Eliminar registros

#### Por atípicos

In [6]:
a_eliminar = df.loc[df.ingresos > 300000].index.values

In [7]:
df = df[~df.index.isin(a_eliminar)]

### Seleccionar variables

Quedarse solo con las de la lista.

In [8]:
df = df[variables_finales]

## CREAR EL PIPELINE

### Instanciar calidad de datos

#### Crear la función

In [9]:
def calidad_datos(temp):
    
    temp['antigüedad_empleo'] = temp['antigüedad_empleo'].fillna('desconocido')
    
    temp.select_dtypes('number').fillna(0,inplace=True)
    
    return(temp)

### Instanciar creación de variables

Dado que la creación de variables es diferente para los 3 modelos necesitamos construir 3 funciones.

#### Crear las funciones

In [10]:
def creacion_variables_pd(df):
    
    temp = df.copy()
    
    temp['target_pd'] = np.where(temp.estado.isin(['Charged Off','Does not meet the credit policy. Status:Charged Off','Default']), 1, 0)
    
    temp.vivienda = temp.vivienda.replace(['ANY','NONE','OTHER'],'MORTGAGE')
    
    temp.finalidad = temp.finalidad.replace(['wedding','educational','renewable_energy'],'otros')
    
    #Eliminamos las variables que ya no usaremos
    temp.drop(columns = ['estado','imp_amortizado','imp_recuperado'],inplace = True)
    
    #Separamos entre predictoras y target
    temp_x = temp.iloc[:,:-1]
    temp_y = temp.iloc[:,-1]
    
    return(temp_x,temp_y)

In [11]:
def creacion_variables_ead(df):
    
    temp = df.copy()
    
    temp['pendiente'] = temp.principal - temp.imp_amortizado
    
    temp['target_ead'] = temp.pendiente / temp.principal
    
    temp.vivienda = temp.vivienda.replace(['ANY','NONE','OTHER'],'MORTGAGE')
    
    temp.finalidad = temp.finalidad.replace(['wedding','educational','renewable_energy'],'otros')
    
    #Eliminamos las variables que ya no usaremos
    temp.drop(columns = ['estado','imp_amortizado','imp_recuperado','pendiente'],inplace = True)
    
    #Separamos entre predictoras y target
    temp_x = temp.iloc[:,:-1]
    temp_y = temp.iloc[:,-1]
    
    return(temp_x,temp_y)

In [12]:
def creacion_variables_lgd(df):
    
    temp = df.copy()
    
    temp['pendiente'] = temp.principal - temp.imp_amortizado
    
    temp['target_lgd'] = 1 - (temp.imp_recuperado / temp.pendiente)
    
    temp['target_lgd'].fillna(0,inplace=True)
    
    temp.vivienda = temp.vivienda.replace(['ANY','NONE','OTHER'],'MORTGAGE')
    
    temp.finalidad = temp.finalidad.replace(['wedding','educational','renewable_energy'],'otros')
    
    #Eliminamos las variables que ya no usaremos
    temp.drop(columns = ['estado','imp_amortizado','imp_recuperado','pendiente'],inplace = True)
    
    #Separamos entre predictoras y target
    temp_x = temp.iloc[:,:-1]
    temp_y = temp.iloc[:,-1]
    
    return(temp_x,temp_y)

#### Crear los dataframes de X e y

In [13]:
x_pd, y_pd = creacion_variables_pd(calidad_datos(df))

x_ead, y_ead = creacion_variables_ead(calidad_datos(df))

x_lgd, y_lgd = creacion_variables_lgd(calidad_datos(df))

### Instanciar transformación de variables

In [14]:
#ONE HOT ENCODING
var_ohe = [ 'ingresos_verificados', 'vivienda','finalidad','num_cuotas']
ohe = OneHotEncoder(sparse = False, handle_unknown='ignore')


#ORDINAL ENCODING
var_oe = ['antigüedad_empleo','rating']

orden_antigüedad_empleo = ['desconocido','< 1 year','1 year','2 years','3 years','4 years',
                           '5 years','6 years','7 years','8 years','9 years','10+ years']

orden_rating = ['A','B','C','D','E','F','G']

oe = OrdinalEncoder(categories = [orden_antigüedad_empleo,orden_rating],
                    handle_unknown = 'use_encoded_value',
                    unknown_value = 12)

#BINNING
var_bin = ['num_derogatorios']
bina = Binarizer(threshold=0)


#MIN-MAX SCALING
var_mms = ['ingresos','dti','num_lineas_credito','porc_uso_revolving',
            'principal','tipo_interes','imp_cuota']
mms = MinMaxScaler()


### Crear el pipe del preprocesamiento

#### Crear el column transformer

In [15]:
ct = make_column_transformer(
    (ohe, var_ohe),
    (oe, var_oe),
    (bina, var_bin),
    (mms, var_mms),
    remainder='passthrough')

### Instanciar los modelos

#### Instanciar los algoritmos

In [16]:
modelo_pd = LogisticRegression(solver = 'saga', n_jobs=-1, C = 0.25, penalty = 'l1')

modelo_ead = HistGradientBoostingRegressor(learning_rate = 0.1,
                                          max_iter = 200,
                                          max_depth = 10,
                                          min_samples_leaf = 100,
                                          scoring = 'neg_mean_absolute_percentage_error',
                                          l2_regularization = 0.75)

modelo_lgd = HistGradientBoostingRegressor(learning_rate = 0.1,
                                          max_iter = 200,
                                          max_depth = 20,
                                          min_samples_leaf = 100,
                                          scoring = 'neg_mean_absolute_percentage_error',
                                          l2_regularization = 0)

#### Crear los pipes finales de entrenamiento

In [17]:
pipe_entrenamiento_pd = make_pipeline(ct,modelo_pd)

pipe_entrenamiento_ead = make_pipeline(ct,modelo_ead)

pipe_entrenamiento_lgd = make_pipeline(ct,modelo_lgd)

#### Guardar el pipe final de entrenamiento

In [18]:
ruta_pipe_entrenamiento_pd = ruta_proyecto + '/04_Modelos/pipe_entrenamiento_pd.pickle'

with open(ruta_pipe_entrenamiento_pd, mode='wb') as file:
   pickle.dump(pipe_entrenamiento_pd, file)

In [19]:
ruta_pipe_entrenamiento_ead = ruta_proyecto + '/04_Modelos/pipe_entrenamiento_ead.pickle'

with open(ruta_pipe_entrenamiento_ead, mode='wb') as file:
   pickle.dump(pipe_entrenamiento_ead, file)

In [20]:
ruta_pipe_entrenamiento_lgd = ruta_proyecto + '/04_Modelos/pipe_entrenamiento_lgd.pickle'

with open(ruta_pipe_entrenamiento_lgd, mode='wb') as file:
   pickle.dump(pipe_entrenamiento_lgd, file)

#### Entrenar los pipes

In [21]:
pipe_ejecucion_pd = pipe_entrenamiento_pd.fit(x_pd,y_pd)
pipe_ejecucion_ead = pipe_entrenamiento_ead.fit(x_ead,y_ead)
pipe_ejecucion_lgd = pipe_entrenamiento_lgd.fit(x_lgd,y_lgd)

## GUARDAR EL PIPE

### Guardar el pipe final de ejecución

In [22]:
ruta_pipe_ejecucion_pd = ruta_proyecto + '/04_Modelos/pipe_ejecucion_pd.pickle'

with open(ruta_pipe_ejecucion_pd, mode='wb') as file:
   pickle.dump(pipe_ejecucion_pd, file)

In [23]:
ruta_pipe_ejecucion_ead = ruta_proyecto + '/04_Modelos/pipe_ejecucion_ead.pickle'

with open(ruta_pipe_ejecucion_ead, mode='wb') as file:
   pickle.dump(pipe_ejecucion_ead, file)

In [24]:
ruta_pipe_ejecucion_lgd = ruta_proyecto + '/04_Modelos/pipe_ejecucion_lgd.pickle'

with open(ruta_pipe_ejecucion_lgd, mode='wb') as file:
   pickle.dump(pipe_ejecucion_lgd, file)